# Simulation with traffic generation.

This demo is for the case of extrapolating the maximum effect caused by a regional traffic flow. 

The bridge: 

- Has four 3.5m-width lanes. 
- Has a length of 20m.
- Has a width of 16m.
- Has one load effect being considered. 

The traffic flow: 

- Vehicles are generated from Garage model. 
- Headways are in freeflow condition (Poisson arrival model). 



---

The part that has already been introduced in the single_vehicle demo. 

But this time we change to define a 4-lane bridge. 

In [1]:
import pybtls as pb
from pathlib import Path


# set the load effect by using a 2D influence surface
lanes_position = [(0.5, 4.0), (4.0, 7.5), (8.5, 12.0), (12.0, 15.5)]
IS_matrix = [
    [0.0, 0.0, 4.0, 8.0, 12.0, 16.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [10.0, 0.0, 2.5, 5.0, 2.5, 0.0],
    [20.0, 0.0, 0.0, 0.0, 0.0, 0.0],
]
load_effect = pb.InfluenceSurface()
load_effect.set_IS(IS_matrix, lanes_position)

# set the bridge
bridge = pb.Bridge(length=20.0, no_lane=4)
bridge.add_load_effect(inf_line_surf=load_effect, threshold=0.0)
bridge.add_load_effect(inf_line_surf=load_effect, threshold=1000.0)  # We add the same load effect again, but this time with a threshold of 1000.0 for the later POT analysis

Set lane flow compositions (here we just use the same flow for all lanes). 

For the lane_dir, 1 means from left to right and 2 means from right to left, 
based on the same coordinate system as the influence surface. 

In [2]:
lfc_lane1 = pb.LaneFlowComposition(lane_index=1, lane_dir=1)
lfc_lane1.assign_lane_data(
    hourly_truck_flow=[100] * 24,
    hourly_car_flow=[0] * 24,
    hourly_speed_mean=[80 / 3.6 * 10] * 24,
    hourly_speed_std=[10.0] * 24,
)

lfc_lane2 = pb.LaneFlowComposition(lane_index=2, lane_dir=1)
lfc_lane2.assign_lane_data(
    hourly_truck_flow=[100] * 24,
    hourly_car_flow=[0] * 24,
    hourly_speed_mean=[80 / 3.6 * 10] * 24,
    hourly_speed_std=[10.0] * 24,
)

lfc_lane3 = pb.LaneFlowComposition(lane_index=3, lane_dir=2)
lfc_lane3.assign_lane_data(
    hourly_truck_flow=[100] * 24,
    hourly_car_flow=[0] * 24,
    hourly_speed_mean=[80 / 3.6 * 10] * 24,
    hourly_speed_std=[10.0] * 24,
)

lfc_lane4 = pb.LaneFlowComposition(lane_index=4, lane_dir=2)
lfc_lane4.assign_lane_data(
    hourly_truck_flow=[100] * 24,
    hourly_car_flow=[0] * 24,
    hourly_speed_mean=[80 / 3.6 * 10] * 24,
    hourly_speed_std=[10.0] * 24,
)

Set vehicle generator. Here we use the garage one.


In [3]:
# The kernel is to ensure variation between the generated vehicles and garages.
kernel = [[1.0, 0.08], [1.0, 0.05], [1.0, 0.02]]

vehicle_gen = pb.VehicleGenGarage(
    garage=Path(".") / "garage.txt", kernel=kernel, garage_format=4
)

Choose headway generator with the freeflow one.

In [4]:
# set headway generator
headway_gen = pb.HeadwayGenFreeflow()

Assemble the vehicle generator and the headway generator with the lfc information to get a traffic generator.

In [5]:
traffic_gen = pb.TrafficGenerator(no_lane=4)
traffic_gen.add_lane(vehicle_gen=vehicle_gen, headway_gen=headway_gen, lfc=lfc_lane1)
traffic_gen.add_lane(vehicle_gen=vehicle_gen, headway_gen=headway_gen, lfc=lfc_lane2)
traffic_gen.add_lane(vehicle_gen=vehicle_gen, headway_gen=headway_gen, lfc=lfc_lane3)
traffic_gen.add_lane(vehicle_gen=vehicle_gen, headway_gen=headway_gen, lfc=lfc_lane4)
# traffic_gen.set_start_time(0.0)  # optional, the default start_time is 0.0 unless you want to change it.

Set output, which will be written to HDD. 

To do the extrapolation, we need the block-max and peak-over-threshold output. 

In [6]:
output_config = pb.OutputConfig()
output_config.set_BM_output(
    write_vehicle=True, write_summary=True, write_mixed=True
)
output_config.set_POT_output(
    write_vehicle=True, write_summary=True, write_counter=True
)


Set the simulation and run. 

In [7]:
sim_task = pb.Simulation(Path(".") / "temp")
sim_task.add_sim(
    bridge=bridge,
    traffic=traffic_gen,
    no_day=250,  # 1-year working days
    output_config=output_config,
    time_step=0.1,
    min_gvw=35,  # Ignore vehicles with GVW less than 35 kN. 
    # active_lane=[1,2,3,4],  # optional, if not set, all lanes will be active.
    # track_progress=False,  # optional, if True, the progress print will show up.
    tag="Case2",
)

# run simulation
sim_task.run(no_core=1)


Check what are the outputs.

In [8]:
sim_output = sim_task.get_output()
print(sim_output.keys())


dict_keys(['Case2'])


Use two simple loops to display all the read-in outputs. 

The data are stored as pandas.DataFrame.

In [9]:
from IPython.display import display

case2_output = sim_output["Case2"]

for i in case2_output.get_summary():
    print(i)

    for j in case2_output.read_data(i):
        print(j)
        display(case2_output.read_data(i)[j])
        

BM_by_no_trucks
BM_V_20_2


,Index,Effect,Value,Time,Position on Bridge,No. Trucks,Trucks
0,1,1,851.5,86009.7,28.40,2,[<pybtls.lib.BTLS.Vehicle object at 0x127ed317...
1,1,2,851.5,86009.7,28.40,2,[<pybtls.lib.BTLS.Vehicle object at 0x127ed323...
2,2,1,912.2,141409.0,26.90,2,[<pybtls.lib.BTLS.Vehicle object at 0x127ed3cb...
3,2,2,912.2,141409.0,26.90,2,[<pybtls.lib.BTLS.Vehicle object at 0x127ed347...
4,3,1,728.1,210254.1,-2.57,2,[<pybtls.lib.BTLS.Vehicle object at 0x127ed3dd...
...,...,...,...,...,...,...,...
495,248,2,848.5,21373964.1,21.78,2,[<pybtls.lib.BTLS.Vehicle object at 0x14010f1d...
496,249,1,509.6,21489013.1,14.24,2,[<pybtls.lib.BTLS.Vehicle object at 0x14010f29...
497,249,2,509.6,21489013.1,14.24,2,[<pybtls.lib.BTLS.Vehicle object at 0x14010f35...
498,250,1,716.2,21575517.0,15.06,2,[<pybtls.lib.BTLS.Vehicle object at 0x14010f41...


BM_V_20_1


,Index,Effect,Value,Time,Position on Bridge,No. Trucks,Trucks
0,1,1,1152.8,72178.8,22.32,1,[<pybtls.lib.BTLS.Vehicle object at 0x127edb8f0>]
1,1,2,1152.8,72178.8,22.32,1,[<pybtls.lib.BTLS.Vehicle object at 0x127edb950>]
2,2,1,1152.6,89180.0,21.65,1,[<pybtls.lib.BTLS.Vehicle object at 0x127edb830>]
3,2,2,1152.6,89180.0,21.65,1,[<pybtls.lib.BTLS.Vehicle object at 0x127edaf90>]
4,3,1,1142.6,175921.2,22.02,1,[<pybtls.lib.BTLS.Vehicle object at 0x127edae10>]
...,...,...,...,...,...,...,...
495,248,2,1122.3,21400190.1,-1.61,1,[<pybtls.lib.BTLS.Vehicle object at 0x140164d70>]
496,249,1,1116.6,21508482.4,22.69,1,[<pybtls.lib.BTLS.Vehicle object at 0x140164dd0>]
497,249,2,1116.6,21508482.4,22.69,1,[<pybtls.lib.BTLS.Vehicle object at 0x140164e30>]
498,250,1,1148.9,21522860.1,22.02,1,[<pybtls.lib.BTLS.Vehicle object at 0x140164e90>]


BM_V_20_3


...

,Peak Index,Time,No. Trucks,Peak Value
0,1,15308.3,1,1076.6
1,2,19863.9,1,1044.7
2,3,31969.8,1,1149.6
3,4,33774.4,1,1109.9
4,5,65469.4,1,1062.8
...,...,...,...,...
2182,10,21575323.8,1,1078.3
2183,11,21576665.2,1,1102.4
2184,12,21587461.9,1,1073.3
2185,13,21593103.8,1,1006.2


PT_S_20_Eff_1


,Peak Index,Time,No. Trucks,Peak Value
0,1,77.8,1,174.2
1,2,122.2,1,151.2
2,3,160.6,1,277.0
3,4,166.3,1,698.5
4,5,214.7,1,207.2
...,...,...,...,...
372655,2656,21599843.5,1,51.5
372656,2657,21599859.9,1,603.5
372657,2658,21599860.3,1,557.9
372658,2659,21599867.3,1,204.4


POT_counter
PT_C_20


,Block,Effect 1,Effect 2
0,1,1624,8
1,2,1435,7
2,3,1560,8
3,4,1546,8
4,5,1602,10
...,...,...,...
282,247,1392,10
283,248,1519,11
284,249,322,1
285,249,1195,2


The simulation output can be saved and loaded for further analysis.

In [10]:
pb.save_output(sim_output, Path("./temp") / "output.pkl")
sim_output = pb.load_output(Path("./temp") / "output.pkl")

Outputs have been successfully saved to temp/output.pkl!
Outputs have been successfully loaded from temp/output.pkl!
